In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.keras.models import Model, Sequential, clone_model, model_from_json
from tensorflow.keras.layers import Input, Dense, Concatenate, Activation, Layer,LeakyReLU
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras import backend as K


import random
from datetime import datetime
import gym
import time


from collections import deque
import time
from datetime import datetime
#from keras.utils.vis_utils import plot_model
import sys
import os

In [4]:
action_space = [-2. , -1.6, -1.2, -0.8, -0.4,  0. ,  0.4,  0.8,  1.2,  1.6,  2. ]
def epsilon_greedy(model, state, eps):
    p = np.random.random()
    if p < (1 - max(eps, 0.05)): # Burada mutlaka keşfetmeyi sağlama amacıyla yapıldı
        max_index = np.argmax(model.predict(state,verbose=0))
        return np.array([action_space[max_index]]), max_index
    else:
        random_index = np.random.choice(np.arange(len(action_space)))
        return np.array([action_space[random_index]]), random_index

In [7]:
#env = gym.make("Pendulum-v1")
#
#state = env.reset()
#
#state = np.reshape(state , [1,state_size])
#epsilon_greedy(DQNetworkPendulum, state, 1)
#a,_ = epsilon_greedy(DQNetworkPendulum, state, 1)

# Model

In [1]:
if not os.path.isdir("Log"):
    os.mkdir("Log")

memory = deque(maxlen=10000) 
min_experience = 1000
batch_size = 64
np.random.seed(124)


env = gym.make("Pendulum-v1")

action_space = [-2. , -1.6, -1.2, -0.8, -0.4,  0. ,  0.4,  0.8,  1.2,  1.6,  2. ]

state_size = len(env.observation_space.sample())
action_size = len(action_space)


#Prediction Model
DQNetworkPendulum = Sequential(name = "DQNetworkPendulum")
DQNetworkPendulum.add(Dense(64, 
                            activation = LeakyReLU(alpha = 0.01),
                            input_shape = (state_size, ),
                            kernel_initializer = 'he_uniform'))
DQNetworkPendulum.add(Dense(32, 
                            activation = LeakyReLU(alpha = 0.01), 
                            kernel_initializer = 'he_uniform'))
DQNetworkPendulum.add(Dense(action_size,
                            activation = 'linear' ))

DQN_optimizer = Adam(learning_rate = 0.01)

DQNetworkPendulum.compile(optimizer = DQN_optimizer, 
                          loss = 'mean_squared_error')



#Target Model
TargetNetworkPendulum = Sequential(name = "DQNetworkPendulum")
TargetNetworkPendulum.add(Dense(64, 
                                activation = LeakyReLU(alpha = 0.01),
                                input_shape = (state_size, ),
                                kernel_initializer = 'he_uniform'))
TargetNetworkPendulum.add(Dense(32, 
                                activation = LeakyReLU(alpha = 0.01), 
                                kernel_initializer = 'he_uniform'))
TargetNetworkPendulum.add(Dense(action_size,
                                activation = 'linear' ))

TargetN_optimizer = Adam(learning_rate = 0.01)

TargetNetworkPendulum.compile(optimizer = TargetN_optimizer, 
                              loss = 'mean_squared_error')




TargetNetworkPendulum.set_weights(DQNetworkPendulum.get_weights())

NameError: name 'os' is not defined

In [ ]:
GAMMA = 0.97
n_episodes = 500
episode_scores = []

for episode in range(0, n_episodes):
    state = env.reset()
    state = np.reshape(state , [1,state_size])
    done = False
    episode_score = 0
    best_score = 0
    epsilon = 0.97 ** episode
    
    start_time_episode = datetime.now()
    while not done:
        action, action_index = epsilon_greedy(DQNetworkPendulum, state, eps = epsilon)
        #time.sleep(0.05)
        #env.render()
        next_state, reward, done, info = env.step(action)
        next_state = np.reshape(next_state , [1,state_size])
        
        #if done and episode_score != (-env._max_episode_steps)+1: # 200. adımda done olduysa yani artık oyun bitti hedefe ulaşamadı o zman -100 ceza ver
        #    reward = 200
        #else: # episodun sonuna gelip done da TRUE
        #    reward = reward
            
        memory.append((state, action_index, reward, next_state, done))
        
        if len(memory) > min_experience:
            minibatch = random.sample(memory, min(len(memory), batch_size ))
            states = np.zeros((batch_size, state_size))
            next_states = np.zeros((batch_size, state_size))
            
            actions, rewards, dones = [], [], []
            
            for i in range(batch_size):
                states[i] = minibatch[i][0]
                actions.append((minibatch[i][1]))
                rewards.append((minibatch[i][2]))
                next_states[i] = minibatch[i][3]
                dones.append((minibatch[i][4]))
                
            targets = DQNetworkPendulum.predict(states, verbose = 0)
            target_nexts = TargetNetworkPendulum.predict(next_states, verbose = 0)
            
            for i in range(batch_size):
                if dones[i]:
                    targets[i][actions[i]] = rewards[i]
                else:
                    targets[i][actions[i]] = rewards[i] + GAMMA * (np.amax(target_nexts[i]))
            
            
            DQNetworkPendulum.fit(states, targets, batch_size = batch_size, verbose = 0)
        state = next_state
        
        episode_score += reward
        
        if done:
            print("episode: {}, score: {}".format(episode, episode_score))
            
    episode_scores.append(episode_score)
    np.savetxt("Log/Episode_scores.txt",episode_scores)
    if (episode + 1)%10 == 0:
        TargetNetworkPendulum.set_weights(DQNetworkPendulum.get_weights())
        print("TargetNetwork equalized to DQNetworkPendulum at {} th episode".format(episode))
        name = "DQNetworkPendulum_episode{}".format(episode)
        DQNetworkPendulum.save(name)
    end_time_episode = datetime.now()
    print("elapse time of {}th episode is: {}".format(episode,(end_time_episode - start_time_episode)))
        
plt.plot(episode_scores)

episode: 0, score: -1473.8373063696692
elapse time of 0th episode is: 0:00:00.060468
episode: 1, score: -1485.6425413490954
elapse time of 1th episode is: 0:00:00.984953
episode: 2, score: -1015.0401255920873
elapse time of 2th episode is: 0:00:00.846829
episode: 3, score: -969.0088101028051
elapse time of 3th episode is: 0:00:01.173495
episode: 4, score: -1324.9759848546996
elapse time of 4th episode is: 0:00:02.305984
episode: 5, score: -972.0450404373522
elapse time of 5th episode is: 0:00:39.210470
episode: 6, score: -900.966755696252
elapse time of 6th episode is: 0:00:35.246241
episode: 7, score: -1225.5120340677245
elapse time of 7th episode is: 0:00:36.231302
episode: 8, score: -999.9290036708284
elapse time of 8th episode is: 0:00:33.845198
episode: 9, score: -1255.2194850032934
TargetNetwork equalized to DQNetworkPendulum at 9 th episode


INFO:tensorflow:Assets written to: DQNetworkPendulum_episode9/assets


INFO:tensorflow:Assets written to: DQNetworkPendulum_episode9/assets


elapse time of 9th episode is: 0:00:34.899362
episode: 10, score: -873.9225468298287
elapse time of 10th episode is: 0:00:34.728609
episode: 11, score: -1013.5498939547722
elapse time of 11th episode is: 0:00:46.054251
episode: 12, score: -1186.3146814543006
elapse time of 12th episode is: 0:00:48.285860
episode: 13, score: -1030.7287689401344
elapse time of 13th episode is: 0:00:35.495651
episode: 14, score: -1153.7924975163949
elapse time of 14th episode is: 0:00:47.234528
episode: 15, score: -1078.5516123301504
elapse time of 15th episode is: 0:00:44.790701
episode: 16, score: -1095.180810285963
elapse time of 16th episode is: 0:00:39.790839
episode: 17, score: -1243.7726082202432
elapse time of 17th episode is: 0:00:42.116457
episode: 18, score: -1359.565978323231
elapse time of 18th episode is: 0:00:39.539090
episode: 19, score: -1037.1553682351664
TargetNetwork equalized to DQNetworkPendulum at 19 th episode


INFO:tensorflow:Assets written to: DQNetworkPendulum_episode19/assets


...

INFO:tensorflow:Assets written to: DQNetworkPendulum_episode379/assets


elapse time of 379th episode is: 0:00:51.234606
episode: 380, score: -363.61284961090803
elapse time of 380th episode is: 0:00:50.276277
episode: 381, score: -234.15321914855866
elapse time of 381th episode is: 0:00:51.827318
episode: 382, score: -122.63458624381225
elapse time of 382th episode is: 0:00:52.303272
episode: 383, score: -126.90146498433843
elapse time of 383th episode is: 0:00:53.583396
episode: 384, score: -124.96737839039955
elapse time of 384th episode is: 0:01:16.263480
episode: 385, score: -123.8557156683999
elapse time of 385th episode is: 0:00:53.592533
episode: 386, score: -123.24618054221936
elapse time of 386th episode is: 0:00:57.474369
episode: 387, score: -5.189924731058778
elapse time of 387th episode is: 0:01:05.434938
episode: 388, score: -478.184819641698
elapse time of 388th episode is: 0:00:58.743767
episode: 389, score: -125.51824060933077
TargetNetwork equalized to DQNetworkPendulum at 389 th episode


INFO:tensorflow:Assets written to: DQNetworkPendulum_episode389/assets


INFO:tensorflow:Assets written to: DQNetworkPendulum_episode389/assets


elapse time of 389th episode is: 0:01:02.240504
episode: 390, score: -237.45577865784102
elapse time of 390th episode is: 0:01:01.132697
episode: 391, score: -128.01984320802208
elapse time of 391th episode is: 0:00:53.287068
episode: 392, score: -123.1610251438562
elapse time of 392th episode is: 0:00:51.738245
episode: 393, score: -232.6444312790808
elapse time of 393th episode is: 0:00:50.147370
episode: 394, score: -243.9123624507026
elapse time of 394th episode is: 0:00:51.135541


# Watch Agent

In [5]:
action_space = [-2. , -1.6, -1.2, -0.8, -0.4,  0. ,  0.4,  0.8,  1.2,  1.6,  2. ]
def epsilon_greedy(model, state, eps):
    p = np.random.random()
    if p < (1 - max(eps, 0.05)): # Burada mutlaka keşfetmeyi sağlama amacıyla yapıldı
        max_index = np.argmax(model.predict(state,verbose=0))
        return np.array([action_space[max_index]]), max_index
    else:
        random_index = np.random.choice(np.arange(len(action_space)))
        return np.array([action_space[random_index]]), random_index

In [6]:
env = gym.make("Pendulum-v1")


state_size = len(env.observation_space.sample())
action_size = env.action_space.sample().shape[0]

In [7]:
def watch_agent(model):
    done = False
    episode_reward = 0
    s = env.reset()
    s = np.reshape(s,[1,state_size])
    while not done:
        a, action_index = epsilon_greedy(DQNetworkPendulum, s, eps = 0)
        s,r,done,info=env.step(a)
        s = np.reshape(s,[1,state_size])
        time.sleep(0.05)
        env.render()
        episode_reward +=r
    print("Episode reward: ",episode_reward)

In [8]:
DQNetworkPendulum = tf.keras.models.load_model("DQNetworkPendulum_episode389")

In [9]:
for i in range(3):
    watch_agent(DQNetworkPendulum)
env.close()

Episode reward:  -129.49336723612825
Episode reward:  -456.28802666783633
Episode reward:  -1.6950611242237885


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import RMSprop

import random
from datetime import datetime
import gym

from collections import deque
import time
from datetime import datetime

In [ ]:
def NNModel(input_size, action_size, hidden_layer_sizes, act, model_name):
    model = Sequential(name = model_name)
    model.add(Dense(hidden_layer_sizes[0], input_shape = (input_size,), activation = act, 
                    kernel_initializer='he_uniform'))
    
    if len(hidden_layer_sizes)>1:
        for node_size in hidden_layer_sizes[1:]:
            model.add(Dense(node_size, activation = act,kernel_initializer='he_uniform'))
    model.add(Dense(action_size, activation='linear',kernel_initializer='he_uniform'))
    model.compile(loss='mse', optimizer = RMSprop(learning_rate = 0.00025,epsilon = 0.01))
    
    return model

In [ ]:
def epsilon_greedy(model, state, eps):
    p = np.random.random()
    if p < (1 - max(eps, 0.05)): # Burada mutlaka keşfetmeyi sağlama amacıyla yapıldı
        return np.argmax(model.predict(state,verbose=0))
    else:
        return env.action_space.sample()

In [ ]:
def watch_agent(model):
    done = False
    episode_reward = 0
    s = env.reset()
    s = np.reshape(s,[1,state_size])
    while not done:
        a = epsilon_greedy(model,s,eps=0)
        s,r,done,info=env.step(a)
        s = np.reshape(s,[1,state_size])
        time.sleep(0.05)
        env.render()
        episode_reward -=r
    print("Episode reward: ",episode_reward)

# Deep Q Network with BATCH

In [ ]:
memory = deque(maxlen=2000) 
min_experience = 500
batch_size = 64
np.random.seed(124)


env = gym.make("MountainCar-v0")
env._max_episode_steps = 1000

state_size = len(env.observation_space.sample())
action_size = env.action_space.n


DQNModel =  NNModel(input_size = state_size,
                   action_size = action_size, 
                   hidden_layer_sizes = [600,500, 200],
                   act='tanh',model_name = 'MountainCar')




GAMMA = 0.95
n_episodes = 5000
episode_scores = []

for episode in range(n_episodes):
    state = env.reset()
    state = np.reshape(state , [1,state_size])
    done = False
    episode_score = 0
    best_score = 0
    epsilon = 0.97 ** episode
    
    while not done:
        action = epsilon_greedy(DQNModel, state, eps = epsilon)
        time.sleep(0.05)
        env.render()
        next_state, reward, done, info = env.step(action)
        next_state = np.reshape(next_state , [1,state_size])
        
        if done and episode_score != (-env._max_episode_steps)+1: # 200. adımda done olduysa yani artık oyun bitti hedefe ulaşamadı o zman -100 ceza ver
            reward = 100
        else: # episodun sonuna gelip done da TRUE
            reward = reward
            
        memory.append((state, action, reward, next_state, done))
        
        if len(memory) > min_experience:
            minibatch = random.sample(memory, min(len(memory), batch_size ))
            states = np.zeros((batch_size, state_size))
            next_states = np.zeros((batch_size, state_size))
            
            actions, rewards, dones = [], [], []
            
            for i in range(batch_size):
                states[i] = minibatch[i][0]
                actions.append((minibatch[i][1]))
                rewards.append((minibatch[i][2]))
                next_states[i] = minibatch[i][3]
                dones.append((minibatch[i][4]))
                
            targets = DQNModel.predict(states,verbose=0)
            target_nexts = DQNModel.predict(next_states,verbose=0)
            
            for i in range(batch_size):
                if dones[i]:
                    targets[i][actions[i]] = rewards[i]
                else:
                    targets[i][actions[i]] = rewards[i] + GAMMA* (np.amax(target_nexts[i]))
            
            
            DQNModel.fit(states, targets, batch_size = batch_size, verbose=0)
        state = next_state
        
        episode_score = episode_score-1
        if done:
            print("episode: {}, score: {}".format(episode, episode_score))
            
        episode_scores.append(episode_score)
plt.plot(episode_scores)

In [ ]:
DQNModel.save("DQNCarBatchNoTarget")

In [ ]:
model = tf.keras.models.load_model("DQNCarBatchNoTarget")

for i in range(3):
    watch_agent(model)
env.close()

# Deep Q Network with BATCH and Target 

In [ ]:
"DQNmodel_episode19"

In [ ]:
19

In [ ]:
memory = deque(maxlen=20000) 
min_experience = 2000
batch_size = 128
np.random.seed(124)


env = gym.make("MountainCar-v0")
env._max_episode_steps = 400

state_size = len(env.observation_space.sample())
action_size = env.action_space.n


#DQNModel =  NNModel(input_size = state_size,
#                    action_size = action_size, 
#                    hidden_layer_sizes = [200, 200, 200],
#                    act='relu',
#                    model_name = 'MountainCar')

DQNModel = tf.keras.models.load_model("DQNmodel_episode49")

TargetNetwork =  NNModel(input_size = state_size,
                         action_size = action_size, 
                         hidden_layer_sizes = [200, 200, 200],
                         act='relu',
                         model_name = 'MountainCar')

TargetNetwork.set_weights(DQNModel.get_weights())

In [ ]:
GAMMA = 0.95
n_episodes = 500
episode_scores = []

for episode in range(49, n_episodes):
    state = env.reset()
    state = np.reshape(state , [1,state_size])
    done = False
    episode_score = 0
    best_score = 0
    epsilon = 0.97 ** episode
    
    start_time_episode = datetime.now()
    while not done:
        action = epsilon_greedy(DQNModel, state, eps = epsilon)
        #time.sleep(0.05)
        #env.render()
        next_state, reward, done, info = env.step(action)
        next_state = np.reshape(next_state , [1,state_size])
        
        if done and episode_score != (-env._max_episode_steps)+1: # 200. adımda done olduysa yani artık oyun bitti hedefe ulaşamadı o zman -100 ceza ver
            reward = 200
        else: # episodun sonuna gelip done da TRUE
            reward = reward
            
        memory.append((state, action, reward, next_state, done))
        
        if len(memory) > min_experience:
            minibatch = random.sample(memory, min(len(memory), batch_size ))
            states = np.zeros((batch_size, state_size))
            next_states = np.zeros((batch_size, state_size))
            
            actions, rewards, dones = [], [], []
            
            for i in range(batch_size):
                states[i] = minibatch[i][0]
                actions.append((minibatch[i][1]))
                rewards.append((minibatch[i][2]))
                next_states[i] = minibatch[i][3]
                dones.append((minibatch[i][4]))
                
            targets = DQNModel.predict(states, verbose = 0)
            target_nexts = TargetNetwork.predict(next_states, verbose = 0)
            
            for i in range(batch_size):
                if dones[i]:
                    targets[i][actions[i]] = rewards[i]
                else:
                    targets[i][actions[i]] = rewards[i] + GAMMA * (np.amax(target_nexts[i]))
            
            
            DQNModel.fit(states, targets, batch_size = batch_size, verbose = 0)
        state = next_state
        
        episode_score = episode_score-1
        
        if done:
            print("episode: {}, score: {}".format(episode, episode_score))
            
    episode_scores.append(episode_score)
    if (episode + 1)%50 == 0:
        TargetNetwork.set_weights(DQNModel.get_weights())
        print("TargetNetwork equalized to DQNModel at {} th episode".format(episode))
        name = "DQNmodel_episode{}".format(episode)
        DQNModel.save(name)
    end_time_episode = datetime.now()
    print("elapse time of {}th episode is: {}".format(episode,(end_time_episode - start_time_episode)))
        
plt.plot(episode_scores)

# Soft Update

In [ ]:
memory = deque(maxlen=20000) 
min_experience = 2000
batch_size = 128
np.random.seed(124)


env = gym.make("MountainCar-v0")
env._max_episode_steps = 400

state_size = len(env.observation_space.sample())
action_size = env.action_space.n


DQNModel =  NNModel(input_size = state_size,
                    action_size = action_size, 
                    hidden_layer_sizes = [200, 200, 200],
                    act='tanh',
                    model_name = 'MountainCar')

TargetNetwork =  NNModel(input_size = state_size,
                         action_size = action_size, 
                         hidden_layer_sizes = [200, 200, 200],
                         act='tanh',
                         model_name = 'MountainCar')

TargetNetwork.set_weights(DQNModel.get_weights())

TAU = 0.05



GAMMA = 0.95
n_episodes = 500
episode_scores = []

for episode in range(n_episodes):
    state = env.reset()
    state = np.reshape(state , [1,state_size])
    done = False
    episode_score = 0
    best_score = 0
    epsilon = 0.97 ** episode
    
    start_time_episode = datetime.now()
    while not done:
        action = epsilon_greedy(DQNModel, state, eps = epsilon)
        #time.sleep(0.05)
        #env.render()
        next_state, reward, done, info = env.step(action)
        next_state = np.reshape(next_state , [1,state_size])
        
        if done and episode_score != (-env._max_episode_steps)+1: # 200. adımda done olduysa yani artık oyun bitti hedefe ulaşamadı o zman -100 ceza ver
            reward = 100
        else: # episodun sonuna gelip done da TRUE
            reward = reward
            
        memory.append((state, action, reward, next_state, done))
        
        if len(memory) > min_experience:
            minibatch = random.sample(memory, min(len(memory), batch_size ))
            states = np.zeros((batch_size, state_size))
            next_states = np.zeros((batch_size, state_size))
            
            actions, rewards, dones = [], [], []
            
            for i in range(batch_size):
                states[i] = minibatch[i][0]
                actions.append((minibatch[i][1]))
                rewards.append((minibatch[i][2]))
                next_states[i] = minibatch[i][3]
                dones.append((minibatch[i][4]))
                
            targets = DQNModel.predict(states, verbose = 0)
            target_nexts = TargetNetwork.predict(next_states, verbose = 0)
            
            for i in range(batch_size):
                if dones[i]:
                    targets[i][actions[i]] = rewards[i]
                else:
                    targets[i][actions[i]] = rewards[i] + GAMMA* (np.amax(target_nexts[i]))
            
            
            DQNModel.fit(states, targets, batch_size = batch_size, verbose=0)
            
            # Soft Update
            q_model_theta = DQNModel.get_weights()
            target_model_theta = TargetNetwork.get_weights()
            
            
            w_counter = 0
            for q_weight, target_weight in zip(q_model_theta, target_model_theta):
                target_weight = target_weight * (1-TAU) + q_weight*TAU
                target_model_theta[w_counter] = target_weight
                w_counter +=1
            
            TargetNetwork.set_weights(target_model_theta)
        state = next_state
        episode_score = episode_score-1
        if done:
            print("episode: {}, score: {}".format(episode, episode_score))     
    episode_scores.append(episode_score)
    if (episode + 1)%50 == 0:
        name = "SoftUpdate/DQNmodelSoftUpdate_episode{}".format(episode)
        DQNModel.save(name)
    end_time_episode = datetime.now()
    print("elapse time of {}th episode is: {}".format(episode,(end_time_episode - start_time_episode)))
        
plt.plot(episode_scores)